# Multilayer Perceptron -  Manual - Cross Validation

### Atualização dos pesos por iteração

### Importação das bibliotecas necessárias

In [ ]:
import pandas as pd
from random import randint
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import explained_variance_score

### Carregamento da base

In [ ]:
NomeBase = "GLCM"
NomeBase = "GLCMMaior"
#NomeBase = "faces"

valor = 1
base = pd.read_csv('../Datasets/' + str(NomeBase)+'.txt', header=None)

entradas = base.iloc[:, 0:len(base.columns) -valor].values 
classes = base.iloc[:,len(base.columns)-valor].values
NumColunas = len(base.columns) - valor
print(classes)
caminho = '../resultados/MLP/PesosMLP_'+str(NomeBase)

In [ ]:
base.head()

In [ ]:
base.describe()

### Padronização dos dados

In [ ]:
for i in range (len(base.columns)-valor):
    entradas[:,i] = (entradas[:,i] - np.mean(entradas[:,i]))/entradas[:,i].std(axis = 0)

### Transformação das classes para o tipo dummy

In [ ]:
labelEncoder = LabelEncoder()
classes = labelEncoder.fit_transform(classes)


### Plotagem dos dados (colunas 1 e 2)

In [ ]:
a1 = 1
a2 = 2

x = base.iloc[:,a1].values
y = base.iloc[:,a2].values

color= ['lime' if l == 0 else 'teal' if l == 1 else 'green' for l in classes]
plt.rcParams['figure.figsize'] = (7,5)
plt.title("Atributos "+ str(a1) + " e " + str(a2) )
plt.scatter(x, y, color=color)
plt.show()

In [ ]:
x = entradas.T
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (20,20)
for i in range(NumColunas):
    plt.subplot(5, 5, i+1)

    sns.distplot(x[i])
plt.show()

In [ ]:
# Simetria
#base.skew()

In [ ]:
# Variância
np.var(base) 

In [ ]:
pltpandas = pd.plotting.boxplot(base.iloc[:, 0:len(base.columns) -valor])
plt.show()

In [ ]:
sns.set(style="ticks")

atrib = np.zeros([NumColunas])
for i in range(NumColunas):
    atrib[i] = int(i)

sns.pairplot(base, hue=NumColunas, vars=atrib)
plt.show()

In [ ]:
base.corr().style.background_gradient()


In [ ]:
# Covariância
print(np.cov(base))

### Divisão da base em treinamento (80%) e teste (20%)

In [ ]:
def dividir_trei_test(entradas, classes, percentual):
    result = []
    while (len(result)) != len(entradas):
        r = randint(0, len(entradas)-1)
        if r not in result:
            result.append(r)    

    repeticoes = np.zeros(np.max(classes)+1)
    
    for i in range(len(entradas)):
        repeticoes[classes[i]] = repeticoes[classes[i]] + 1
    repeticoes = np.array(repeticoes)
    minimo = int(np.min(repeticoes) * (1 - percentual))
    
    treino = int((minimo) * (np.max(classes)+1) )
    teste = int(len(entradas) - treino)

    entradas_trei = np.zeros((treino, NumColunas))
    entradas_test = np.zeros((teste, NumColunas))
    classes_trei = np.zeros((treino)) 
    classes_test = np.zeros((teste))

 
    contTreino = 0
    contTest = 0
    vetorMinimo = np.zeros(np.max(classes)+1)

    for i in range(len(classes)):    
        if (vetorMinimo[classes[result[i]]] < minimo):            
            entradas_trei[contTreino] = entradas[result[i]]
            classes_trei[contTreino] = classes[result[i]]
   
            contTreino = contTreino + 1
            vetorMinimo[classes[result[i]]] = vetorMinimo[classes[result[i]]] + 1
            
        else:   
            entradas_test[contTest] = entradas[result[i]]            
            classes_test[contTest] = classes[result[i]]
           
            contTest = contTest + 1        


    return entradas_trei, entradas_test, classes_trei, classes_test

In [ ]:
labelEncoder = LabelEncoder()
classes = labelEncoder.fit_transform(classes)
NumClasses = np.max(classes) + 2

entradas_trei, entradas_test, classes_t, classes_te = dividir_trei_test(entradas, classes, 0.05)
contador = np.zeros((3))

classes_test =  np.array(pd.get_dummies(classes_te))
classes_trei =  np.array(pd.get_dummies(classes_t))

### Métodos e demais variáveis necessárias

In [ ]:
def funcaoAtivacao(soma, Usu_func):
    if (Usu_func == 1):
        return 1/(1 + np.exp(-soma))
    if (Usu_func == 2):
        return ((1 - np.exp(-soma))/(1 + np.exp(-soma)))
    return 0

In [ ]:
def derivadaParcial(camadaSaida, Usu_func):
    if (Usu_func == 1):
        return camadaSaida * (1 - camadaSaida)
    if (Usu_func == 2):
        return 1/2 * (1 - (camadaSaida)**2)

In [ ]:
def decaimentoTaxaAprend(realizar, tipo, taxaAprendIni, iteracaoAtual, NumIteracoes):
    if (realizar == 1):
        if (tipo == 1):
            # Linear
            return taxaAprendIni * (1 - (iteracaoAtual/NumIteracoes))            
        elif (tipo == 2):
            # Exponencial
             return taxaAprendIni/(1 + iteracaoAtual)  
    else:
        return taxaAprendIni

In [ ]:
def CalcVar(camadaSaida, corretas):    
    classesN = [np.argmax(t) for t in corretas]
    classesPreditas = np.zeros((camadaSaida.shape[0]))
    corretas = np.ones((camadaSaida.shape[0]))
    for i in range(camadaSaida.shape[0]):         
        classesPreditas[i] =  camadaSaida[i][classesN[i]]
    return np.var(classesPreditas)   

## Dados do usuário

In [ ]:
NumClasses = np.max(classes) + 2
epocasAnt = 0

carregarPesos = int(input("Deseja carregar pesos salvos sem continuar a iteração? (1 - Sim, 2 - Não) "))

if (carregarPesos == 1):
        carregarPesCon = 0
        arquivo = input("Digite o número do diretório: ")  
        epocas = np.load(str(str(caminho)+"/"+str(arquivo)+"/epocas.npy"))
        taxaAprendOcul = np.load(str(str(caminho)+"/"+str(arquivo)+"/taxaAprendOcul.npy"))
        decaimentoOcul = np.load(str(str(caminho)+"/"+str(arquivo)+"/decaimentoOcul.npy"))
        tipoDecaimentoOcul = np.load(str(str(caminho)+"/"+str(arquivo)+"/tipoDecaimentoOcul.npy")) 
        taxaAprendSai = np.load(str(str(caminho)+"/"+str(arquivo)+"/taxaAprendSai.npy"))         
        decaimentoSai = np.load(str(str(caminho)+"/"+str(arquivo)+"/decaimentoSai.npy")) 
        tipoDecaimentoSai = np.load(str(str(caminho)+"/"+str(arquivo)+"/tipoDecaimentoSai.npy")) 
        escolha = np.load(str(str(caminho)+"/"+str(arquivo)+"/escolha.npy")) 
        NeuroniosCamadaOculta = np.load(str(str(caminho)+"/"+str(arquivo)+"/NeuroniosCamadaOculta.npy")) 
        Usu_func = np.load(str(str(caminho)+"/"+str(arquivo)+"/Usu_func.npy")) 
        TaxaRejeicao = np.load(str(str(caminho)+"/"+str(arquivo)+"/TaxaRejeicao.npy")) 
        PesoBiasOculta = np.load(str(str(caminho)+"/"+str(arquivo)+"/PesoBiasOculta.npy")) 
        pesos0 = np.load(str(str(caminho)+"/"+str(arquivo)+"/pesos0.npy")) 
        pesos1 = np.load(str(str(caminho)+"/"+str(arquivo)+"/pesos1.npy")) 
        Erro = np.load(str(str(caminho)+"/"+str(arquivo)+"/Erro.npy")) 
        variancia = np.load(str(str(caminho)+"/"+str(arquivo)+"/variancia.npy"))
        K = np.load(str(str(caminho)+"/"+str(arquivo)+"/K.npy"))
        mediaErro = np.load(str(str(caminho)+"/"+str(arquivo)+"/mediaErro.npy"))
        menorErro = np.load(str(str(caminho)+"/"+str(arquivo)+"/menorErro.npy"))
        melhorEpoca = np.load(str(str(caminho)+"/"+str(arquivo)+"/melhorEpoca.npy"))
        mediaErroTest_Val = np.load(str(str(caminho)+"/"+str(arquivo)+"/mediaErroTest_Val.npy"))
      #  DP = np.load(str(str(caminho)+"/"+str(arquivo)+"/DP.npy"))

        
if (carregarPesos != 1):    
    carregarPesCon = int(input("Deseja carregar pesos e continuar com o processamento? (1 - Sim, 2 - Não) "))
    
    dimensoes = NumColunas
    
    if (carregarPesCon == 1):
        print("Atenção! Esta não é uma boa prática, a mesma pode ocasionar vícios na rede.")
        arquivo = input("Digite o número do diretório: ")    
        epocasAnt =  np.load(str("../resultados/PesosMLP_"+str(NomeBase)+"/"+str(arquivo)+"/epocas.npy"))

        epocas = int(input('\033[1m' + "Digite a quantidade de épocas: "))
        epocasAnt = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/epocas.npy"))
        taxaAprendOcul = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/taxaAprendOcul.npy"))
        decaimentoOcul = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/decaimentoOcul.npy"))
        tipoDecaimentoOcul = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/tipoDecaimentoOcul.npy")) 
        taxaAprendSai = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/taxaAprendSai.npy")) 
        
        decaimentoSai = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/decaimentoSai.npy")) 
        tipoDecaimentoSai = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/tipoDecaimentoSai.npy")) 
        escolha = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/escolha.npy")) 
        NeuroniosCamadaOculta = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/NeuroniosCamadaOculta.npy")) 
        Usu_func = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/Usu_func.npy")) 
        TaxaRejeicao = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/TaxaRejeicao.npy")) 
        PesoBiasOculta = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/PesoBiasOculta.npy")) 
        pesos0 = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/pesos0.npy")) 
        pesos1 = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/pesos1.npy")) 
        Erro = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/Erro.npy")) 
        variancia = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/variancia.npy")) 
        K = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/K.npy"))
        mediaErro = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/mediaErro.npy"))
        menorErro = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/menorErro.npy"))
        melhorEpoca = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/melhorEpoca.npy"))
        mediaErroTest_Val = np.load(str(str(caminho)+""+str(NomeBase)+"/"+str(arquivo)+"/mediaErroTest_Val.npy"))
        
    else:
        epocas = int(input('\033[1m' + "Digite a quantidade de épocas: "))
        taxaAprendOcul = float(input('\033[1m' + "Digite a taxa de aprendizado da camada oculta: "))
        decaimentoOcul = int(input('\033[1m' + "Deseja realizar decaimento da taxa de aprendizagem da camada Oculta? (1 - Sim ou 2 - Não): "))
        if (decaimentoOcul == 1):
            print('\033[1m' + "Digite: ")
            print('\033[1m' + "1- Decaimento Linear: ")
            print('\033[1m' + "2- Decaimento exponencial: ")
            tipoDecaimentoOcul = int(input(""))
        else:
            tipoDecaimentoOcul = 0
        taxaAprendSai = float(input('\033[1m' + "Digite a taxa de aprendizado da camada de saída: "))
        taxaAprendSaiIni = taxaAprendSai
        decaimentoSai = int(input('\033[1m' + "Deseja realizar decaimento da taxa de aprendizagem da camada de Saída? (1 - Sim ou 2 - Não): "))
        if (decaimentoSai == 1):
            print('\033[1m' + "Digite: ")
            print('\033[1m' + "1- Decaimento Linear: ")
            print('\033[1m' + "2- Decaimento exponencial: ")
            tipoDecaimentoSai = int(input(""))
        else:
            tipoDecaimentoSai = 0
        print('\033[1m' + "Digite a quantidade de Neurônios da camada oculta: ")
        print("1- Regra do Valor Médio")
        print("2- Regra de Kolmogorov")
        print("3- Regra de Fletcher-Gloss")
        print("4- Regra da Raiz quadrada")
        print("5- Inserir um valor manualmente")
        escolha = int(input(""))
        if (escolha == 1):
                # Regra do valor médio
                qtde =  int((NumClasses + NumColunas)/2)
        if (escolha == 2):
                # Regra de Kolmogorov
                qtde =  int((2 * NumColunas) + 1)
        if (escolha == 3):
                # Regra de Fletcher-Gloss
                a = int((2* (NumColunas ** (1/2)) + NumClasses))
                b = 2 * NumColunas + 1
                qtde = int(randint(a, b))      
        if (escolha == 4):
                # Regra da raiz quadrada
                qtde = int((NumClasses * NumColunas)**(1/2))
        if (escolha == 5):
                qtde = int(input("Digite a quantidade de neurônios que deseja inserir"))
        NeuroniosCamadaOculta = qtde
        print(NeuroniosCamadaOculta)
        print('\033[1m' + "Função de ativação: ")
        print("1- Logística")
        print("2- Tangente Hiperbólica")
        Usu_func = int(input("Digite o valor: "))
        TaxaRejeicao = float(input("Digite o valor para a Faixa de Rejeição: "))        
        print('\033[1m' + "Validação Cruzada - K-fold ")
        K = int(input("Digite a quantidade de partes para a Validação Cruzada: "))
        
        PesoBiasOculta = np.dot((-0.5 + (0.5 + 0.5)), np.random.random((NeuroniosCamadaOculta,1)))
        MelhorPeso0 = np.zeros((NumColunas,NeuroniosCamadaOculta))
        MelhorPeso1 = np.zeros(NeuroniosCamadaOculta, NumClasses)
        
        if (Usu_func == 1):        
            pesos0 = np.dot((-0.5 + (0.5 + 0.5)), np.random.random((NumColunas,NeuroniosCamadaOculta)))
            pesos1 = np.dot((-0.5 + (0.5 + 0.5)), np.random.random((NeuroniosCamadaOculta, NumClasses)))
        else:
            pesos0 = np.dot((-1.45 + (1.45 + 1.45)), np.random.random((NumColunas,NeuroniosCamadaOculta)))
            pesos1 = np.dot((-1.45 + (1.45 + 1.45)), np.random.random((NeuroniosCamadaOculta, NumClasses)))
            
        menorErro = 10000
        melhorEpoca = epocas
        
taxaAprendOculIni = taxaAprendOcul
taxaAprendSaiIni = taxaAprendSai

In [ ]:


classes_trei = np.array(classes_trei, dtype=float)
classes_test = np.array(classes_test, dtype=float)

if (Usu_func == 2):
    for i in range (classes_trei.shape[0]):
        if (classes_trei[i][0] == 0):
            classes_trei[i][0] = classes_trei[i][0] -1
        if (classes_trei[i][1] == 0):
            classes_trei[i][1] = classes_trei[i][1] -1
            
    for i in range (classes_test.shape[0]):
        if (classes_test[i][0] == 0):
            classes_test[i][0] = classes_test[i][0] -1
        if (classes_test[i][1] == 0):
            classes_test[i][1] = classes_test[i][1] -1



### Validação Cruzada

In [ ]:
def SepararTeste(a, inicio, fim, tamanho, NumColunas):
    aux = 0
    x = np.zeros((tamanho, NumColunas))    
    while (inicio < fim):
        x[aux] = a[inicio]
        aux = aux + 1
        inicio = inicio + 1
    return(x)

In [ ]:
def SepararTreino(a, inicio, fim, tamanho, NumColunas):
    aux = 0
    x = np.zeros(((len(a) - tamanho), NumColunas))    
    for i in range(len(a)):
        if ((i < inicio) or (i >= fim)):
            x[aux] = a[i]
            aux = aux + 1
    return(x)

### Acrescenta uma coluna para os valores da Taxa de Rejeição

In [ ]:
temp = classes_trei
num_entradas_trei = len(entradas_trei)
classes_trei = np.zeros((num_entradas_trei, NumClasses))

for i in range(num_entradas_trei):
    for j in range(NumClasses-1):
        classes_trei[i][j] = temp[i][j]  
    
camadaSaida = np.zeros((num_entradas_trei, NumClasses))

## Treinamento

In [ ]:
if (carregarPesos != 1):
    mediaErroTest_Val = np.zeros(epocas)
   # menorErro = np.zeros(epocas + 1)
    
    mediaErro = np.zeros(epocas)    
    variancia = np.zeros(epocas)
    Erro = np.zeros(num_entradas_trei)    
    
    for i in range(epocas):            
        tamanho = int(len(entradas_trei)/K)
        inicio = 0
        fim = tamanho
        camadaSaida_Val = np.zeros((tamanho, NumClasses))
        #print("epoca " + str(i))
        
        for m in range(K): 
           # print("K " + str(m))
            entradas_trei_val = SepararTreino(entradas_trei, inicio, fim, tamanho, NumColunas)
            classes_trei_val = SepararTreino(classes_trei, inicio, fim, tamanho, NumClasses)
            
            entradas_val = SepararTeste(entradas_trei, inicio, fim, tamanho, NumColunas)
            classes_val = SepararTeste(classes_trei, inicio, fim, tamanho, NumClasses)
            
            # iteração
            for j in range(entradas_trei_val.shape[0]):
            
                # Feed Foward
                entrada = entradas_trei_val[j].reshape(1,NumColunas)
                somaSinapse0 = np.dot(entrada, pesos0)        
                camadaOculta = funcaoAtivacao(somaSinapse0 - PesoBiasOculta.T, Usu_func)  
                soma = np.dot(camadaOculta, pesos1)
                camadaSaida[j]  = funcaoAtivacao(soma, Usu_func)

                # Back Propagation
                erroCamadaSaida = classes_trei_val[j] - camadaSaida[j]
                Erro[j] =  np.mean(np.abs(erroCamadaSaida))
                
                # Atualização dos pesos                
                derivadaSaida  = derivadaParcial(camadaSaida[j], Usu_func)
                GradienteSaida = derivadaSaida * erroCamadaSaida
                GradienteSaida = GradienteSaida.reshape(1,NumClasses) 
                pesos1 = pesos1 + taxaAprendSai * camadaOculta.T * GradienteSaida 
                DerivadaOculta = derivadaParcial(camadaOculta, Usu_func)
                GradienteOculta = (DerivadaOculta.T * (np.dot(pesos1, GradienteSaida.T)))
                pesos0 = pesos0 + taxaAprendOcul * (entrada.T * GradienteOculta.T);        

                # Decaimento da taxa de Aprendizagem
                taxaAprendOcul = decaimentoTaxaAprend(decaimentoOcul, tipoDecaimentoOcul, taxaAprendOculIni, j, num_entradas_trei)
                taxaAprendSai = decaimentoTaxaAprend(decaimentoSai, tipoDecaimentoSai, taxaAprendSaiIni, j, num_entradas_trei)
                
                                
            # Validação
           # mediaErroTest_Val = np.zeros(entradas_val.shape[0])
            for n in range(entradas_val.shape[0]):     
                    # Feed Foward
                entrada_v = entradas_val[n].reshape(1, NumColunas)
                somaSinapse0_v = np.dot(entrada_v, pesos0)        
                camadaOculta_v = funcaoAtivacao(somaSinapse0_v + PesoBiasOculta.T, Usu_func)        
                camadaSaida_Val[n] = funcaoAtivacao(np.dot(camadaOculta_v, pesos1), Usu_func)

                # Variância
                varianciaVal = CalcVar(classes_val, camadaSaida_Val)
              #  DPVal = (varianciaVal/varianciaVal.shape) ** (1/2)
              #  print(DPVal)

                # cálculo do erro
                erroCamadaSaida_Val = classes_val - camadaSaida_Val
                
                # Média absoluta do erro
                                
            # Atualização dos campos para a validação cruzada
            inicio = inicio + tamanho
            fim = tamanho + fim
            ErroK = np.mean(np.abs(Erro))
            
        mediaErroTest_Val[i] = np.mean(np.abs(erroCamadaSaida_Val))

        if (mediaErroTest_Val[i] < menorErro):
            menorErro = mediaErroTest_Val[i]
            melhorEpoca = i
            MelhorPeso0 = pesos0
            MelhorPeso1 = pesos1        
        print("Época atual: "+ str(i)+ " de " + str(epocas))
        print("Melhor época: "+ str(melhorEpoca))
        print("Menor Erro: "+ str(menorErro))
        print("Erro na validação: "+ str(mediaErroTest_Val[i]))
   
                
        mediaErro[i] = np.mean(np.abs(ErroK)) 
        variancia[i] = CalcVar(classes_trei, camadaSaida)  
        DP = (variancia/variancia.shape) ** (1/2)
             

    valor  = 0
    import os
    

    if not os.path.exists(str(caminho)+"/"):
        os.mkdir(str(caminho))      
    
    

    if not os.path.exists(str(caminho)+"/"+str(int(valor))):
        os.mkdir(str(caminho)+ "/" + str(int(valor)))    

    else:       
        b = os.listdir(caminho)
        a = np.zeros(len(b))

        for i in range(len(b)):
            a[i] = int(b[i])

        valor = np.max(a)
        valor = valor + 1

        os.mkdir(str(caminho) + "/"+ str(int(valor)))
    
    np.save(str(caminho) + "/"+ str(int(valor))+"/epocas", epocas)
    np.save(str(caminho) + "/"+ str(int(valor))+"/taxaAprendOcul", taxaAprendOcul)
    np.save(str(caminho) + "/"+ str(int(valor))+"/decaimentoOcul", decaimentoOcul)
    np.save(str(caminho) + "/"+ str(int(valor))+"/tipoDecaimentoOcul", tipoDecaimentoOcul)
    np.save(str(caminho) + "/"+ str(int(valor))+"/taxaAprendSai", taxaAprendSai)
    np.save(str(caminho) + "/"+ str(int(valor))+"/decaimentoSai", decaimentoSai)
    np.save(str(caminho) + "/"+ str(int(valor))+"/tipoDecaimentoSai", tipoDecaimentoSai)
    np.save(str(caminho) + "/"+ str(int(valor))+"/escolha", escolha)
    np.save(str(caminho) + "/"+ str(int(valor))+"/NeuroniosCamadaOculta", NeuroniosCamadaOculta)
    np.save(str(caminho) + "/"+ str(int(valor))+"/Usu_func", Usu_func)
    np.save(str(caminho) + "/"+ str(int(valor))+"/TaxaRejeicao", TaxaRejeicao)
    np.save(str(caminho) + "/"+ str(int(valor))+"/PesoBiasOculta", PesoBiasOculta)
    np.save(str(caminho) + "/"+ str(int(valor))+"/pesos0", MelhorPeso0)
    np.save(str(caminho) + "/"+ str(int(valor))+"/pesos1", MelhorPeso1)
    np.save(str(caminho) + "/"+ str(int(valor))+"/Erro", Erro)
    np.save(str(caminho) + "/"+ str(int(valor))+"/variancia", variancia)
    np.save(str(caminho) + "/"+ str(int(valor))+"/K", K)
    np.save(str(caminho) + "/"+ str(int(valor))+"/mediaErro", mediaErro)
    np.save(str(caminho) + "/"+ str(int(valor))+"/menorErro", menorErro)
    np.save(str(caminho) + "/"+ str(int(valor))+"/melhorEpoca", melhorEpoca)
    np.save(str(caminho) + "/"+ str(int(valor))+"/mediaErroTest_Val", mediaErroTest_Val)

In [ ]:
print(variancia)
print(DP)

In [ ]:
plt.rcParams['figure.figsize'] = (20, 16)
plt.subplot(3, 2, 1)
plt.title("Erro Médio do Treinamento")
plt.plot(mediaErro, color='blue')   

plt.subplot(3, 2, 2)
plt.title("Variância")
plt.plot(variancia, color='black')


plt.subplot(3, 2, 3)
plt.title("Erro Médio do Teste ")
plt.plot(mediaErroTest_Val, color='green')


plt.subplot(3, 2, 4)
plt.title("Desvio padrão ")
plt.plot(DP, color='red')
    
    
print(melhorEpoca)
print(menorErro)
print("Variância: "+ str(variancia[melhorEpoca]))
plt.subplot(3, 2, 5)
plt.title("Menor Erro e melhor época")
plt.plot(mediaErro, color='blue')
plt.plot(mediaErroTest_Val, color='green')
plt.scatter(melhorEpoca, menorErro, s=100, color='green')
plt.legend(['Erro Treino', 'Erro Teste', 'Menor erro'], loc='upper left')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (10, 7)
plt.title("Menor Erro e melhor época")
plt.plot(mediaErro, color='blue')
plt.plot(mediaErroTest_Val, color='green')
plt.scatter(melhorEpoca, menorErro, s=100, color='red')
plt.legend(['Erro Treino', 'Erro Teste', 'Menor erro'], loc='upper left')
plt.show()

## Resultados obtidos com os pesos finais

In [ ]:
camadaSaidaTrei = np.zeros((num_entradas_trei, NumClasses))

varianciaTrei = np.zeros((num_entradas_trei))

for j in range(num_entradas_trei):     
    # Feed Foward
    entrada = entradas_trei[j].reshape(1, NumColunas)
    somaSinapse0 = np.dot(entrada, pesos0)        
    camadaOculta = funcaoAtivacao(somaSinapse0 + PesoBiasOculta.T, Usu_func)        
    camadaSaidaTrei[j]  = funcaoAtivacao(np.dot(camadaOculta, pesos1), Usu_func)
    
# Variância
varianciaTrei = CalcVar(classes_trei, camadaSaidaTrei)
    
# cálculo do erro
erroCamadaSaida = classes_trei - camadaSaidaTrei

# Média absoluta do erro
mediaErroTrei = np.mean(np.abs(erroCamadaSaida))
print("Média Absoluta do Erro: "+ str(mediaErroTrei))
print("Variância: "+ str(varianciaTrei))

### Taxa de Rejeição

In [ ]:
for i in range(len(camadaSaidaTrei)):
    a = camadaSaidaTrei[i]
    b = np.sort(a)   
    if ((b[len(b)-1] - b[len(b)-2]) <= TaxaRejeicao):        
        for j in range (camadaSaidaTrei.shape[1]-1):
            camadaSaidaTrei[i][j] = 0
        camadaSaidaTrei[i][camadaSaidaTrei.shape[1]-1] = 1

In [ ]:
mediaErroTrein = str(np.mean(np.abs(mediaErroTrei)))

### Organização das Saídas para plotagem da Matriz de Confusão

In [ ]:
classesN = [np.argmax(t) for t in classes_trei]
SaidasCalcTrein =  [np.argmax(t) for t in camadaSaidaTrei]

classesPreditas = np.zeros((camadaSaidaTrei.shape[0]))

for i in range(camadaSaidaTrei.shape[0]):         
    classesPreditas[i] =  camadaSaidaTrei[i][classesN[i]]
    
print("Variância entre as saídas calculadas "+ str(np.var(classesPreditas)))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de Confusão',
                          cmap=plt.cm.Blues):   

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Saída correta')
    plt.xlabel('Saída encontrada')    

In [ ]:
errosTrei = []
rejeiTrei = []
cont = 0
for i in range (len(classesN)):
    if (classesN[i] != SaidasCalcTrein[i] and SaidasCalcTrein[i] != 2):
        cont += 1


print({x:errosTrei.count(x) for x in set(errosTrei)})
print ("Total de erros "+str(cont))
        
cont = 0
for i in range (len(classesN)):
    if (SaidasCalcTrein[i] == 2):
        cont += 1

        
print({x:rejeiTrei.count(x) for x in set(rejeiTrei)})
print ("Faixa de rejeição "+str(cont))


In [ ]:
plt.rcParams['figure.figsize'] = (7,5)
sns.set_style("dark")
cm = confusion_matrix(classesN, SaidasCalcTrein)

if (carregarPesos != 1):
    np.save(str(caminho) + "/"+ str(int(valor))+"/errosTrei", errosTrei)
    np.save(str(caminho) + "/"+ str(int(valor))+"/rejeiTrei", rejeiTrei)
    np.save(str(caminho) + "/"+ str(int(valor))+"/MC_treino", cm)
#print(cm)
plot_confusion_matrix(cm, 
                      classes=['0', '1','FR'],
                      title='Treinamento')
plt.show()

### Taxa de Erro retirando os valores que entraram na taxa de rejeição

In [ ]:
quantTotal = 0;
NClasses = NumClasses - 2

for i in range(len(SaidasCalcTrein)):
    if (SaidasCalcTrein[i] <= NClasses):
        quantTotal += 1
erros = 0

for i in range (len(SaidasCalcTrein)):
    if ((SaidasCalcTrein[i] != classesN[i]) and (SaidasCalcTrein[i] <= NClasses)):
        erros += 1

Taxa_de_erro_Trei =  (erros * 100)/quantTotal
acerto_trei = 100 - Taxa_de_erro_Trei
print("Taxa de erro desconsiderando as saídas que entraram na taxa de rejeição: "+str(Taxa_de_erro_Trei))
print("Percentual de acerto: "+ str(100 - Taxa_de_erro_Trei))

## Testes

### Acrescenta uma coluna para os valores da Taxa de Rejeição

In [ ]:
num_entradas_test = len(entradas_test)
tempTest = classes_test
classes_test = np.zeros((num_entradas_test, NumClasses))

for i in range(num_entradas_test):
    for j in range(NumClasses-1):
        classes_test[i][j] = tempTest[i][j]  

In [ ]:
camadaSaidaTest = np.zeros((num_entradas_test, NumClasses))
varianciaTest = np.zeros((num_entradas_test))

for j in range(num_entradas_test):     
    # Feed Foward
    entrada = entradas_test[j].reshape(1,NumColunas)
    somaSinapse0 = np.dot(entrada, pesos0)        
    camadaOculta = funcaoAtivacao(somaSinapse0 + PesoBiasOculta.T, Usu_func)        
    camadaSaidaTest[j]  = funcaoAtivacao(np.dot(camadaOculta, pesos1), Usu_func)
    
# Variância
varianciaTest = CalcVar(classes_test, camadaSaidaTest)
    
# cálculo do erro
erroCamadaSaida = classes_test - camadaSaidaTest

# Média absoluta do erro
mediaErroTest = np.mean(np.abs(erroCamadaSaida))
print("Média Absoluta do Erro: "+ str(mediaErroTest))
print("Variância: "+ str(varianciaTest))

### Taxa de Rejeição

In [ ]:
for i in range(len(camadaSaidaTest)):
    a = camadaSaidaTest[i]
    b = np.sort(a)   
    if ((b[len(b)-1] - b[len(b)-2]) <= TaxaRejeicao):        
        for j in range (camadaSaidaTest.shape[1]-1):
            camadaSaidaTest[i][j] = 0
        camadaSaidaTest[i][camadaSaidaTest.shape[1]-1] = 1

### Organização das Saídas para plotagem da Matriz de Confusão

In [ ]:
classesN = [np.argmax(t) for t in classes_test]

In [ ]:
SaidasCalcTest =  [np.argmax(t) for t in camadaSaidaTest]

In [ ]:
errosTest = []
rejeiTest = []
cont = 0
for i in range (len(classesN)):
    if (classesN[i] != SaidasCalcTest[i] and SaidasCalcTest[i] != 2):
        cont += 1


print({x:errosTest.count(x) for x in set(errosTest)})
print ("Total de erros "+str(cont))
        
cont = 0
for i in range (len(classesN)):
    if (SaidasCalcTest[i] == 2):
        cont += 1

        
print({x:rejeiTest.count(x) for x in set(rejeiTest)})
print ("Faixa de rejeição "+str(cont))

In [ ]:
cm = confusion_matrix(classesN, SaidasCalcTest)

if (carregarPesos != 1):
    np.save(str(caminho) + "/"+ str(int(valor))+"/errosTest", errosTest)
    np.save(str(caminho) + "/"+ str(int(valor))+"/rejeiTest", rejeiTest)
    np.save(str(caminho) + "/"+ str(int(valor))+"/MC_teste", cm)

print(cm)
plot_confusion_matrix(cm, 
                      classes=['0', '1', 'FR'],
                      title='Teste')
plt.show()

### Taxa de Erro retirando os valores que entraram na taxa de rejeição

In [ ]:
quantTotal = 0;
NClasses = NumClasses - 2

for i in range(len(SaidasCalcTest)):
    if (SaidasCalcTest[i] <= NClasses):
        quantTotal += 1
erros = 0

for i in range (len(SaidasCalcTest)):
    if ((SaidasCalcTest[i] != classesN[i]) and (SaidasCalcTest[i] <= NClasses)):
        erros += 1

Taxa_de_erro_Test =  (erros * 100)/quantTotal
acerto_test = 100 - Taxa_de_erro_Test
print("Taxa de erro desconsiderando as saídas que entraram na taxa de rejeição: "+str(Taxa_de_erro_Test))
print("Percentual de acerto: "+ str(100 - Taxa_de_erro_Test))

### Criar DataSet das Informações sobre as execuções

#### Abre o arquivo

In [ ]:
if (carregarPesos != 1):  
    caminhoTexto = '../resultados/MLP/'+str(NomeBase)+'Dados.txt'
    
    if not os.path.exists(caminhoTexto):
        arquivo = open(caminhoTexto, 'w')
        arquivo.writelines("epocas, taxaAprendOculIni, taxaAprendOcul, decaimentoOcul, tipoDecaimentoOcul, taxaAprendSaiIni, taxaAprendSai, decaimentoSai, tipoDecaimentoSai, escolha, NeuroniosCamadaOculta, Usu_func, TaxaRejeicao, K-folds, VarianciaTrei, VarianciaTest, MediaErroTrei, MediaErroTest, Taxa_de_acerto_Trei, Taxa_de_acerto_Test, melhorEpoca")
        arquivo.close()     
    
    
    
    
    arquivo = open(caminhoTexto, 'r')
    conteudo = arquivo.readlines()
        
    conteudo.append("\n"+str(epocas+epocasAnt)+", "+ str(taxaAprendOculIni)+", "+ str(taxaAprendOcul)+", "+ str(decaimentoOcul)+", "+ str(tipoDecaimentoOcul)+", "+ str(taxaAprendSaiIni) +", "+ str(taxaAprendSai)+", "+ str(decaimentoSai)+", "+ str(tipoDecaimentoSai)+","+ str(escolha) +", "+ str(NeuroniosCamadaOculta)+", "+ str(Usu_func)+", "+str(TaxaRejeicao)+", "+str(K)+", "+ str(variancia[len(variancia)-1])+", "+ str(varianciaTest)+ ", "+str(mediaErroTrein)+", "+str(mediaErroTest)+ ", "+ str(acerto_trei)+ ","+ str(acerto_test)+","+ str(melhorEpoca))

    #### Abre o arquivo como escrita, escreve o conteúdo e fecha o mesmo
        
    arquivo = open('../resultados/MLP/'+str(NomeBase)+'Dados.txt', 'w')
    arquivo.writelines(conteudo)
    arquivo.close()

#### Leitura dos dados existentes

In [ ]:
print('../resultados/'+str(NomeBase)+'Dados.txt')

arquivo = pd.read_csv('../resultados/MLP/'+str(NomeBase)+'Dados.txt')
arquivo = arquivo.sort_values(by=[' MediaErroTest', ' MediaErroTrei'], ascending=True)
print("10 Melhores Resultados - Base "+ str(NomeBase))
arquivo.head(10)

# Resultados obtidos

In [ ]:
print("\n Épocas: \t \t \t \t \t \t \t \t"+str(epocas+epocasAnt)+
      " \n Taxa de Aprendizado Inicial da Camada Oculta: \t \t \t \t"+ str(taxaAprendOculIni)+
      " \n Taxa de Aprendizado Final da Camada Oculta: \t \t \t \t"+ str(taxaAprendOcul)+
      " \n Decaimento da Taxa de Aprendizado da Camada Oculta: \t \t \t"+ str(decaimentoOcul)+
      " \n Tipo de Decaimento da Taxa de Aprendizado da Camada Oculta: \t  \t"+ str(tipoDecaimentoOcul)+
      " \n Taxa de Aprendizado Inicial da Camada de Saída:  \t \t \t"+ str(taxaAprendSaiIni) +
      " \n Taxa de Aprendizado Final da Camada de Saída:  \t \t \t"+ str(taxaAprendSai)+
      " \n Decaimento da Taxa de Aprendizado da Camada de Saída:  \t \t"+ str(decaimentoSai)+
      " \n Tipo de Decaimento da Taxa de Aprendizado da Camada Oculta:  \t \t"+ str(tipoDecaimentoSai)+
      " \n Escolha do método da quantidade de Neurônios da Camada Oculta: \t"+ str(escolha) +
      " \n Quantidade de Neurônios da Camada Oculta: \t \t \t \t"+ str(NeuroniosCamadaOculta)+
      " \n Função de Ativação: \t \t \t \t \t \t \t"+ str(Usu_func)+
      " \n Taxa de Rejeição: \t \t \t \t \t \t \t"+str(TaxaRejeicao)+
      " \n Variância do Treinamento: \t \t \t \t \t \t"+ str(variancia[len(variancia)-1])+
      " \n Variância dos Testes: \t \t \t \t \t \t \t"+ str(varianciaTest)+ 
      " \n Erro Médio do Treinamento: \t \t \t \t \t \t"+str(mediaErroTrein)+
      " \n Erro Médio do Teste: \t \t \t \t \t \t \t"+str(mediaErroTest)+ 
      " \n Taxa de Erro do Treinamento (excluíndo os valores da TR): \t \t"+ str(Taxa_de_erro_Trei)+ 
      " \n Taxa de Erro do Teste (excluíndo os valores da TR): \t \t \t"+ str(Taxa_de_erro_Test))

In [ ]:
def classificacao(entrada):
    entradas2 = np.zeros((entradas.shape[0] + 1, entradas.shape[1]))
  
    for i in range (entradas.shape[0]+1):
        if (i == (entradas.shape[0])):
            entradas2[i] = entrada
           # print(entradas2[i])
        else:
            entradas2[i] =  entradas[i]            
    maximo = i
    a = entradas2[maximo]
    
    for i in range (len(base.columns)-1):
        entradas2[:,i] = (entradas2[:,i] - np.mean(entradas2[:,i]))/entradas2[:,i].std(axis = 0)
    
    
    somaSinapse0 = np.dot(entradas2[maximo], pesos0)        
    camadaOculta = funcaoAtivacao(somaSinapse0 + PesoBiasOculta.T, Usu_func)        
    camadaSaidaTest  = funcaoAtivacao(np.dot(camadaOculta, pesos1), Usu_func)
    return camadaSaidaTest